In [1]:
import datetime

import numpy as np
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
import matplotlib.pyplot as plt
import os






store = pd.HDFStore("/Users/nipunbatra/Downloads/wiki-temp.h5")

a = store.keys()
def num_from_key(key):
    return int(key[1:])

def key_from_num(num):
    return "/"+str(num)


ids = map(num_from_key, a)

cols = ['house_square_feet', "total_annual_income", "number_floors",'sex_males', 'sex_females', 'house_num_rooms', 'year_house_constructed']
from copy import deepcopy
cols_plus_data_id = deepcopy(cols)
cols_plus_data_id.insert(0, "dataid")

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 26 days


In [2]:
ds_path = "/Users/nipunbatra/wikienergy-2013_ipsn.h5"

In [3]:
from nilmtk import *

In [4]:
ds = DataSet(ds_path)

In [5]:
original_name_dict = {b.metadata['original_name']:b.identifier.instance for b in ds.buildings.values()}
original_name_map = pd.Series(original_name_dict)
reverse_name_map = pd.Series({v:k for k,v in original_name_dict.iteritems() })

In [6]:
original_name_map[94]

1

In [7]:
train = DataSet(ds_path)
test = DataSet(ds_path)

In [8]:
from nilmtk.disaggregate import FHMM

In [9]:
def find_specific_appliance(appliance_name, appliance_instance, list_of_elecs):
    for elec_name in list_of_elecs:
        appl = elec_name.appliances[0]
        if (appl.identifier.type, appl.identifier.instance) == (appliance_name, appliance_instance):
            return elec_name

In [10]:
import datetime

import numpy as np
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
import matplotlib.pyplot as plt
import os

#START, STOP = '2013-07-01', '2013-07-31'


from nilmtk.disaggregate.hart_85 import *
from nilmtk.feature_detectors.steady_states import *



store = pd.HDFStore("/Users/nipunbatra/Downloads/wiki-temp.h5")

a = store.keys()
def num_from_key(key):
    return int(key[1:])

def key_from_num(num):
    return "/"+str(num)



wm_homes = [  94,  370,  545,  624,  871, 1697, 2156, 2242, 2470, 2814, 3367,
            3456, 3723, 3778, 3967, 4031, 4767, 4957, 5357, 7510, 7769, 7800,
            9499, 9654, 9729, 9922, 9934, 9982]






from nilmtk import *
ds_path = "/Users/nipunbatra/wikienergy-2013_ipsn.h5"

ds = DataSet(ds_path)

original_name_dict = {b.metadata['original_name']:b.identifier.instance for b in ds.buildings.values()}
original_name_map = pd.Series(original_name_dict)
reverse_name_map = pd.Series({v:k for k,v in original_name_dict.iteritems() })

train = DataSet(ds_path)
test = DataSet(ds_path)



from nilmtk.disaggregate import FHMM

def find_specific_appliance(appliance_name, appliance_instance, list_of_elecs):
    for elec_name in list_of_elecs:
        appl = elec_name.appliances[0]
        if (appl.identifier.type, appl.identifier.instance) == (appliance_name, appliance_instance):
            return elec_name



In [11]:
out = {}

months_start_stop = {
    1: ('2013-01-01', '2013-01-31'),
    2: ('2013-02-01', '2013-02-28'),
    3: ('2013-03-01', '2013-03-31'),
    4: ('2013-04-01', '2013-04-30'),
    5: ('2013-05-01', '2013-05-31'),
    6: ('2013-06-01', '2013-06-30'),
    7: ('2013-07-01', '2013-07-31'),
    8: ('2013-08-01', '2013-08-31'),
    9: ('2013-09-01', '2013-09-30'),
    10: ('2013-10-01', '2013-10-31'),
    11: ('2013-11-01', '2013-11-30'),
    12: ('2013-12-01', '2013-12-31')
}

In [21]:
train = DataSet(ds_path)
train.set_window(start="2013-06-01", end="2013-07-30")
fhmm_out = {}
hart_out = {}

In [27]:

for id_home in wm_homes[:]:
    if id_home in fhmm_out or id_home in hart_out:
        continue
    
    fhmm_out[id_home] = {}
    hart_out[id_home] = {}



    # Train on Hart
    train_power_df = store[key_from_num(id_home)]["2013-07-01":"2013-07-15"]['use']
    train_overall_survey_df = store[key_from_num(id_home)]["2013-07-01":"2013-07-15"]
    hvac_df = train_overall_survey_df['clotheswasher1']
    
    """
    ss_total, tr_total = find_steady_states(pd.DataFrame({"power":train_power_df}))

    from nilmtk.disaggregate import Hart85
    from nilmtk.disaggregate.hart_85 import hart85_means_shift_cluster
    from nilmtk.disaggregate import FHMM
    
    h = Hart85()
    h.transients = deepcopy(tr_total)
    pair_df = h.pair(buffer_size=20,
      min_tolerance=100, percent_tolerance=0.035,
      large_transition=1000)
    h.centroids = hart85_means_shift_cluster(pair_df, cols=[('power','active')])
    d = (h.centroids - hvac_df[hvac_df>1000].mean()).abs()
    ac_num = d[('power','active')].argmin()
    ac_identifier_tuple = ('unknown', ac_num)
    """

    # Train FHMM
    b_id = original_name_map[id_home]
    train_elec = train.buildings[b_id].elec

    top_5_train_elec = train_elec.submeters().select_top_k(k=4)
    hvac_elec_train = train_elec[('washing machine', 1)]
    if hvac_elec_train not in top_5_train_elec.meters:
        top_5_train_elec.meters.append(hvac_elec_train)
    fhmm = FHMM()
    fhmm.train(top_5_train_elec)
    ac_instance=1
    
    
    for month in range(1, 13):
        print "*"*80
        print month
        print "*"*80
        test = DataSet(ds_path)
        test.set_window(start=months_start_stop[month][0], end=months_start_stop[month][1])
        print id_home
        print "--"*80
        try:
            
            overall_survey_df = store[key_from_num(id_home)][months_start_stop[month][0]:months_start_stop[month][1]]
            power_df = overall_survey_df['use']


            

            if power_df.sum()>0 and hvac_df.sum()>0:
                pdf = pd.DataFrame({"power":power_df})

                
                #pred_df_ac_hart = h.disaggregate_chunk(pdf, {}, h.transients)[[ac_num]].squeeze()
                

                


                
                test_elec = test.buildings[b_id].elec
                test_mains = test_elec.mains()
                test_mains_df = test_mains.load().next()
                ac_elec_train = train_elec[('washing machine', 1)]
                ac_elec_test = test_elec[('washing machine', 1)]
                
                pred_df_fhmm = fhmm.disaggregate_chunk(test_mains_df)
                pred_df_ac_fhmm = pred_df_fhmm[[find_specific_appliance('washing machine',
                                                                  ac_instance,
                                                                  pred_df_fhmm.columns.tolist())]]
                pred_ser_ac = pred_df_ac_fhmm.squeeze()
                pred_ser_ac.name = "FHMM"

                fhmm_out[id_home][month] = pred_ser_ac.sum()
                #hart_out[id_home][month] = pred_df_ac_hart.sum()
               





        except Exception, e:
            print e, e.message
            pass


    #pred_df = pd.DataFrame(out)


1/20 ElecMeter(instance=1, building=2, dataset='Dataport', appliances=[Appliance(type='light', instance=1)])

HDF5ExtError: HDF5 error back trace

  File "H5Dio.c", line 173, in H5Dread
    can't read data
  File "H5Dio.c", line 545, in H5D__read
    can't read data
  File "H5Dchunk.c", line 1861, in H5D__chunk_read
    unable to read raw data chunk
  File "H5Dchunk.c", line 2886, in H5D__chunk_lock
    unable to read raw data chunk
  File "H5Fio.c", line 113, in H5F_block_read
    read through metadata accumulator failed
  File "H5Faccum.c", line 258, in H5F_accum_read
    driver read request failed
  File "H5FDint.c", line 142, in H5FD_read
    driver read request failed
  File "H5FDsec2.c", line 692, in H5FD_sec2_read
    addr overflow, addr = 3290282600, size=372, eoa=3290281734

End of HDF5 error back trace

Problems reading records.

In [23]:
train_elec['washing machine'].load().next().describe()

,power
,active
count,84960.000000
mean,2.798599
std,39.529518
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1013.000000


In [24]:
train_elec.appliances

[Appliance(type='fridge', instance=1),
 Appliance(type='dish washer', instance=1),
 Appliance(type='washing machine', instance=1),
 Appliance(type='spin dryer', instance=1),
 Appliance(type='electric furnace', instance=1),
 Appliance(type='sockets', instance=1),
 Appliance(type='waste disposal unit', instance=1),
 Appliance(type='air conditioner', instance=1)]

In [26]:
pd.DataFrame(fhmm_out).T.mul(.000017)

,1,2,3,4,5,6,7,8,9,10,11,12
94,62.389524,32.208234,48.336678,43.729032,71.618994,134.213844,146.384484,91.0146,132.98505,59.271384,45.182022,36.517938


In [25]:
pd.DataFrame(fhmm_out).T.mul(.000017).dropna().to_csv("/Users/nipunbatra/git/monthly-disaggregation/fhmm_disag_new.csv")
#pd.DataFrame(hart_out).T

In [50]:
fhmm.model

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1,
      init_params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      means_prior=None, means_weight=0, n_components=162, n_iter=10,
      params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      random_state=None, startprob=None, startprob_prior=None, thresh=0.01,
      transmat=None, transmat_prior=None)

In [51]:
fhmm.disaggregate_chunk(test_mains_df)

,"ElecMeter(instance=7, building=1, dataset='Dataport', appliances=[Appliance(type='sockets', instance=1)])","ElecMeter(instance=8, building=1, dataset='Dataport', appliances=[Appliance(type='fridge', instance=1)])","ElecMeter(instance=1, building=1, dataset='Dataport', appliances=[Appliance(type='dish washer', instance=1)])","ElecMeter(instance=9, building=1, dataset='Dataport', appliances=[Appliance(type='air conditioner', instance=1)])","ElecMeter(instance=5, building=1, dataset='Dataport', appliances=[Appliance(type='electric furnace', instance=1)])"
localminute,,,,,
2013-03-01 00:00:00-06:00,46,113,0,0,53
2013-03-01 00:01:00-06:00,46,113,58,0,10
2013-03-01 00:02:00-06:00,46,113,58,0,10
2013-03-01 00:03:00-06:00,46,113,58,0,10
2013-03-01 00:04:00-06:00,46,113,58,0,10
2013-03-01 00:05:00-06:00,46,113,58,0,10
2013-03-01 00:06:00-06:00,46,113,58,0,10
2013-03-01 00:07:00-06:00,46,113,58,0,10
2013-03-01 00:08:00-06:00,46,113,58,0,10


In [37]:
months_start_stop[month][1]

'2013-04-31'

In [44]:
test.set_window(start='2013-04-01' ,end='2013-04-31')

ValueError: 